In [ ]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
import shutil
shutil.move(list(uploaded.keys())[0], '/content/filtered_results.csv')
# Read the CSV file into a DataFrame
df = pd.read_csv('/content/filtered_results.csv')
df.head()

Saving filtered_results.csv to filtered_results.csv


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,6/11/2021,Italy,Turkey,3,0,UEFA Euro,Rome,Italy,False
1,6/12/2021,Wales,Switzerland,1,1,UEFA Euro,Cardiff,Wales,False
2,6/12/2021,Denmark,Finland,0,1,UEFA Euro,Copenhagen,Denmark,False
3,6/13/2021,Austria,North Macedonia,3,1,UEFA Euro,Bucharest,Romania,True
4,6/13/2021,Netherlands,Ukraine,3,2,UEFA Euro,Amsterdam,Netherlands,False


In [ ]:
df_germany = df[(df['home_team'] == 'Germany') | (df['away_team'] == 'Germany')]
df_scotland = df[(df['home_team'] == 'Scotland') | (df['away_team'] == 'Scotland')]
df_switzerland = df[(df['home_team'] == 'Switzerland') | (df['away_team'] == 'Switzerland')]
df_hungary= df[(df['home_team'] == 'Hungary') | (df['away_team'] == 'Hungary')]


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
9,6/15/2021,Germany,France,0,1,UEFA Euro,Munich,Germany,False
19,6/19/2021,Germany,Portugal,4,2,UEFA Euro,Munich,Germany,False
28,6/23/2021,Germany,Hungary,2,2,UEFA Euro,Munich,Germany,False
36,6/29/2021,England,Germany,2,0,UEFA Euro,London,England,False
64,9/2/2021,Liechtenstein,Germany,0,2,FIFA World Cup qualification,St. Gallen,Switzerland,True


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
1,6/12/2021,Wales,Switzerland,1,1,UEFA Euro,Cardiff,Wales,False
11,6/16/2021,Italy,Switzerland,3,0,UEFA Euro,Rome,Italy,False
21,6/20/2021,Switzerland,Turkey,3,1,UEFA Euro,Baku,Azerbaijan,True
35,6/28/2021,France,Switzerland,3,3,UEFA Euro,Bucharest,Romania,True
38,7/2/2021,Switzerland,Spain,1,1,UEFA Euro,Saint Petersburg,Russia,True
47,9/1/2021,Switzerland,Greece,2,1,Friendly,Basel,Switzerland,False
79,9/5/2021,Switzerland,Italy,0,0,FIFA World Cup qualification,Basel,Switzerland,False
99,9/8/2021,Northern Ireland,Switzerland,0,0,FIFA World Cup qualification,Belfast,Northern Ireland,False
117,10/9/2021,Switzerland,Northern Ireland,2,0,FIFA World Cup qualification,Geneva,Switzerland,False
139,10/12/2021,Lithuania,Switzerland,0,4,FIFA World Cup qualification,Vilnius,Lithuania,False


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
def predict_score(team1_df, team2_df):
    features = ['date', 'home_team', 'away_team', 'tournament', 'city', 'country', 'neutral']
    target_variables = ['home_score', 'away_score']

    # Concatenate team1 and team2 dataframes
    combined_df = pd.concat([team1_df, team2_df])

    # Reindex the combined dataframe with the specified columns
    combined_df = combined_df.reindex(columns=features + target_variables, fill_value=0)

    # Convert categorical features to dummy variables
    combined_df = pd.get_dummies(combined_df, columns=features)

    # Split data into features and target variables
    X = combined_df.drop(target_variables, axis=1)
    y = combined_df[target_variables]

    # Apply weights to features based on tournament type
    tournament_weights = {'tournament_UEFA EURO': 2, 'tournament_FIFA World Cup': 3}  # Weight for 'Friendly' matches
    for feature, weight in tournament_weights.items():
        if feature in X.columns:
            X[feature] *= weight

    # Model Training
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = LinearRegression()  # You can use any other model of your choice
    model.fit(X_train, y_train)


    # Prepare team1 and team2 dataframes for prediction
    team1_df_prep = team1_df.reindex(columns=X.columns, fill_value=0)
    team2_df_prep = team2_df.reindex(columns=X.columns, fill_value=0)

    # Prediction
    team1_score_prediction = model.predict(team1_df_prep)
    team2_score_prediction = model.predict(team2_df_prep)

    mse_home = mean_squared_error(team1_df['home_score'], team1_score_prediction[:, 0].round())
    mse_away = mean_squared_error(team2_df['away_score'], team2_score_prediction[:, 1].round())

    print("MSE Home:", mse_home)
    print("MSE Away:", mse_away)

    # Round the predicted scores
    team1_score_prediction = team1_score_prediction.round()
    team2_score_prediction = team2_score_prediction.round()




    return team1_score_prediction, team2_score_prediction

# Let's assume 'Friendly' tournaments should have less weight
 # Assign a weight of 0.5 to 'Friendly' tournaments



In [ ]:

# Print the predicted scores along with the home team and away team
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Germany", "Scotland", predict_score(df_germany, df_scotland)))


MSE Home: 4.117647058823529
MSE Away: 1.5666666666666667
Predicted Scores for Future Match - Home Team (Germany) vs Away Team (Scotland): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Hungary", "Switzerland", predict_score(df_hungary, df_switzerland)))

MSE Home: 1.1666666666666667
MSE Away: 2.2
Predicted Scores for Future Match - Home Team (Hungary) vs Away Team (Switzerland): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
   

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Germany", "Hungary", predict_score(df_germany, df_hungary)))

MSE Home: 4.117647058823529
MSE Away: 1.2
Predicted Scores for Future Match - Home Team (Germany) vs Away Team (Hungary): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Scotland", "Switzerland", predict_score(df_scotland, df_switzerland)))

Predicted Scores for Future Match - Home Team (Scotland) vs Away Team (Switzerland): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1.,

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Germany", "Switzerland", predict_score(df_germany, df_switzerland)))

Predicted Scores for Future Match - Home Team (Germany) vs Away Team (Switzerland): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Scotland", "Hungary", predict_score(df_scotland, df_hungary)))

Predicted Scores for Future Match - Home Team (Scotland) vs Away Team (Hungary): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]

In [ ]:
df_spain = df[(df['home_team'] == 'Spain') | (df['away_team'] == 'Spain')]
df_italy = df[(df['home_team'] == 'Italy') | (df['away_team'] == 'Italy')]
df_croatia = df[(df['home_team'] == 'Croatia') | (df['away_team'] == 'Croatia')]
df_albania= df[(df['home_team'] == 'Albania') | (df['away_team'] == 'Albania')]
df_albania

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
82,9/5/2021,Albania,Hungary,1,0,FIFA World Cup qualification,Elbasan,Albania,False
124,10/9/2021,Hungary,Albania,0,1,FIFA World Cup qualification,Budapest,Hungary,False
159,11/12/2021,England,Albania,5,0,FIFA World Cup qualification,London,England,False
209,3/26/2022,Spain,Albania,2,1,Friendly,Barcelona,Spain,False
212,3/29/2022,Albania,Georgia,0,0,Friendly,Tirana,Albania,False
257,6/6/2022,Iceland,Albania,1,1,UEFA Nations League,Reykjavík,Iceland,False
276,6/10/2022,Albania,Israel,1,2,UEFA Nations League,Tirana,Albania,False
296,6/13/2022,Albania,Estonia,0,0,Friendly,Tirana,Albania,False
324,9/24/2022,Israel,Albania,2,1,UEFA Nations League,Tel Aviv,Israel,False
343,9/27/2022,Albania,Iceland,1,1,UEFA Nations League,Tirana,Albania,False


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Spain", "Croatia", predict_score(df_spain, df_croatia)))

MSE Home: 3.611111111111111
MSE Away: 2.4324324324324325
Predicted Scores for Future Match - Home Team (Spain) vs Away Team (Croatia): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Italy", "Albania", predict_score(df_italy, df_albania)))

Predicted Scores for Future Match - Home Team (Italy) vs Away Team (Albania): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Croatia", "Albania", predict_score(df_albania, df_croatia)))

Predicted Scores for Future Match - Home Team (Croatia) vs Away Team (Albania): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Spain", "Italy", predict_score(df_spain, df_italy)))

MSE Home: 3.611111111111111
MSE Away: 1.7428571428571429
Predicted Scores for Future Match - Home Team (Spain) vs Away Team (Italy): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Spain", "Albania", predict_score(df_spain, df_albania)))

Predicted Scores for Future Match - Home Team (Spain) vs Away Team (Albania): (array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.]]), array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Croatia", "Italy", predict_score(df_croatia, df_italy)))

Predicted Scores for Future Match - Home Team (Croatia) vs Away Team (Italy): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
 

In [ ]:
df_england= df[(df['home_team'] == 'England') | (df['away_team'] == 'England')]
df_denmark= df[(df['home_team'] == 'Denmark') | (df['away_team'] == 'Denmark')]
df_slovenia = df[(df['home_team'] == 'Slovenia') | (df['away_team'] == 'Slovenia')]
df_serbia= df[(df['home_team'] == 'Serbia') | (df['away_team'] == 'Serbia')]


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Denmark", "Slovenia", predict_score(df_slovenia, df_denmark)))

Predicted Scores for Future Match - Home Team (Denmark) vs Away Team (Slovenia): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("England", "Serbia", predict_score(df_serbia, df_england)))

Predicted Scores for Future Match - Home Team (England) vs Away Team (Serbia): (array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.]]), array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("England", "Serbia", predict_score(df_slovenia, df_serbia)))

Predicted Scores for Future Match - Home Team (England) vs Away Team (Serbia): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Denmark", "England", predict_score(df_denmark, df_england)))

Predicted Scores for Future Match - Home Team (Denmark) vs Away Team (England): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("England", "Slovenia", predict_score(df_england, df_slovenia)))

Predicted Scores for Future Match - Home Team (England) vs Away Team (Slovenia): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Denmark", "Serbia", predict_score(df_denmark, df_serbia)))

Predicted Scores for Future Match - Home Team (Denmark) vs Away Team (Serbia): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],


In [ ]:
df_belgium = df[(df['home_team'] == 'Belgium') | (df['away_team'] == 'Belgium')]
df_slovakia = df[(df['home_team'] == 'Slovakia') | (df['away_team'] == 'Slovakia')]
df_romania = df[(df['home_team'] == 'Romania') | (df['away_team'] == 'Romania')]
df_ukraine= df[(df['home_team'] == 'Ukraine') | (df['away_team'] == 'Ukraine')]


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Romania", "Ukraine", predict_score(df_romania, df_ukraine)))

Predicted Scores for Future Match - Home Team (Romania) vs Away Team (Ukraine): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Belgium", "Slovakia", predict_score(df_belgium, df_slovakia)))


Predicted Scores for Future Match - Home Team (Belgium) vs Away Team (Slovakia): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Slovakia", "Ukraine", predict_score(df_slovakia, df_ukraine)))

Predicted Scores for Future Match - Home Team (Slovakia) vs Away Team (Ukraine): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Belgium", "Romania", predict_score(df_belgium, df_romania)))

Predicted Scores for Future Match - Home Team (Belgium) vs Away Team (Romania): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Slovakia", "Romania", predict_score(df_slovakia, df_romania)))

Predicted Scores for Future Match - Home Team (Slovakia) vs Away Team (Romania): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Belgium", "Ukraine", predict_score(df_ukraine, df_belgium)))

Predicted Scores for Future Match - Home Team (Belgium) vs Away Team (Ukraine): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]))


In [ ]:
df_turkey = df[(df['home_team'] == 'Turkey') | (df['away_team'] == 'Turkey')]
df_georgia = df[(df['home_team'] == 'Georgia') | (df['away_team'] == 'Georgia')]
df_portugal = df[(df['home_team'] == 'Portugal') | (df['away_team'] == 'Portugal')]
df_czech_republic = df[(df['home_team'] == 'Czech Republic') | (df['away_team'] == 'Czech Republic')]

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Turkey", "Georgia", predict_score(df_turkey, df_georgia)))

Predicted Scores for Future Match - Home Team (Turkey) vs Away Team (Georgia): (array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.]]), array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Portugal", "Czech", predict_score(df_portugal, df_czech_republic)))

Predicted Scores for Future Match - Home Team (Portugal) vs Away Team (Czech): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Georgia", "Czech Republic", predict_score(df_georgia, df_czech_republic)))

Predicted Scores for Future Match - Home Team (Georgia) vs Away Team (Czech Republic): (array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.]]), array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Turkey", "Portugal", predict_score(df_turkey, df_portugal)))

Predicted Scores for Future Match - Home Team (Turkey) vs Away Team (Portugal): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Georgia", "Portugal", predict_score(df_georgia, df_portugal)))

Predicted Scores for Future Match - Home Team (Georgia) vs Away Team (Portugal): (array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.]]), array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.]

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Czech Republic", "Turkey", predict_score(df_czech_republic, df_turkey)))

Predicted Scores for Future Match - Home Team (Czech Republic) vs Away Team (Turkey): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]))


In [ ]:
df_poland = df[(df['home_team'] == 'Poland') | (df['away_team'] == 'Poland')]
df_netherlands = df[(df['home_team'] == 'Netherlands') | (df['away_team'] == 'Netherlands')]
df_austria = df[(df['home_team'] == 'Austria') | (df['away_team'] == 'Austria')]
df_france = df[(df['home_team'] == 'France') | (df['away_team'] == 'France')]

print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Poland", "Netherlands", predict_score(df_poland, df_netherlands)))

MSE Home: 0.8
MSE Away: 2.1176470588235294
Predicted Scores for Future Match - Home Team (Poland) vs Away Team (Netherlands): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Poland", "Netherlands", predict_score(df_austria, df_france)))

Predicted Scores for Future Match - Home Team (Poland) vs Away Team (Netherlands): (array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.]]), array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Austria", "France", predict_score(df_france, df_austria)))

Predicted Scores for Future Match - Home Team (Austria) vs Away Team (France): (array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.]]), array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Poland", "Austria", predict_score(df_poland, df_austria)))

Predicted Scores for Future Match - Home Team (Poland) vs Away Team (Austria): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Netherlands", "france", predict_score(df_netherlands, df_france)))

Predicted Scores for Future Match - Home Team (Netherlands) vs Away Team (france): (array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.]]), array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Netherlands", "Austria", predict_score(df_netherlands, df_austria)))

Predicted Scores for Future Match - Home Team (Netherlands) vs Away Team (Austria): (array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.]]), array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("France", "Poland", predict_score(df_france, df_poland)))

Predicted Scores for Future Match - Home Team (France) vs Away Team (Poland): (array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.]]), array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Spain", "Ukraine", predict_score(df_spain, df_ukraine)))

Predicted Scores for Future Match - Home Team (Spain) vs Away Team (Ukraine): (array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.]]), array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
 

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Switzerland", "Serbia", predict_score(df_switzerland, df_serbia)))

Predicted Scores for Future Match - Home Team (Switzerland) vs Away Team (Serbia): (array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.]]), array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Georgia", "Scotland", predict_score(df_georgia, df_scotland)))

Predicted Scores for Future Match - Home Team (Georgia) vs Away Team (Scotland): (array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.]]), array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.]

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("France", "Romania", predict_score(df_france, df_romania)))

Predicted Scores for Future Match - Home Team (France) vs Away Team (Romania): (array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.]]), array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Slovakia", "Croatia", predict_score(df_slovakia, df_croatia)))

Predicted Scores for Future Match - Home Team (Slovakia) vs Away Team (Croatia): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]))


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Netherlands", "Portugal", predict_score(df_netherlands, df_portugal)))

Predicted Scores for Future Match - Home Team (Netherlands) vs Away Team (Portugal): (array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.]]), array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2.,

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("England", "Turkey", predict_score(df_england, df_turkey)))

Predicted Scores for Future Match - Home Team (England) vs Away Team (Turkey): (array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.]]), array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Germany", "Italy", predict_score(df_germany, df_italy)))

Predicted Scores for Future Match - Home Team (Germany) vs Away Team (Italy): (array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
 

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Spain", "Switzerland", predict_score(df_spain, df_switzerland)))

Predicted Scores for Future Match - Home Team (Spain) vs Away Team (Switzerland): (array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.]]), array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("France", "Georgia", predict_score(df_france, df_georgia)))

Predicted Scores for Future Match - Home Team (France) vs Away Team (Georgia): (array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.]]), array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],


In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Portugal", "Croatia", predict_score(df_portugal, df_croatia)))

Predicted Scores for Future Match - Home Team (Portugal) vs Away Team (Croatia): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Germany", "England", predict_score(df_germany, df_england)))

Predicted Scores for Future Match - Home Team (Germany) vs Away Team (England): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("Spain", "France", predict_score(df_spain, df_france)))

Predicted Scores for Future Match - Home Team (Spain) vs Away Team (France): (array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.]]), array([[2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
       [2., 2.],
  

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("England", "Croatia", predict_score(df_england, df_croatia)))

Predicted Scores for Future Match - Home Team (England) vs Away Team (Croatia): (array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.]]), array([[1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],
       [1., 2.],

In [ ]:
print("Predicted Scores for Future Match - Home Team ({}) vs Away Team ({}): {}".format("France", "Croatia", predict_score(df_france, df_croatia)))

MSE Home: 6.527777777777778
MSE Away: 2.4324324324324325
Predicted Scores for Future Match - Home Team (France) vs Away Team (Croatia): (array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.]]), array([[2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2., 1.],
       [2.,